### Imports

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud

np.set_printoptions(threshold=np.inf)
#nltk.download('punkt')

### Recogemos los datos limpios

In [ ]:
# Cargamos las noticias limpias en df_clean
df_clean = pd.read_csv('../data/train_clean.csv')

### Tokenización

In [ ]:
# Tokenizamos el texto
df_clean_tokens = pd.DataFrame()
df_clean_tokens['text'] = df_clean['text'].apply(nltk.word_tokenize)
df_clean_tokens['label'] = df_clean['label']

### Split train/test

In [ ]:
# Dividimos el dataset en train y test con la función sample de pandas
df_train, df_test = train_test_split(df_clean_tokens, test_size=0.2, random_state=777)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para test: ", len(df_test))

### Etiquetamos texto

In [ ]:
# Si label es 0, incluimos en text True, False si es 1
for i in range(len(df_train['label'])):
	if df_train['label'].iloc[i] == 0:
		df_train['text'].iloc[i].append("TRUE")
	else:
		df_train['text'].iloc[i].append("FALSE")
  
df_train = df_train.drop('label', 1)

### Data Visualization

In [ ]:
# Creamos dos word cloud con el texto limpio y tokenizado de cada label
wordcloud = WordCloud(width = 800, height = 800,
				background_color ='white',
				min_font_size = 10).generate(str(df_clean_tokens['text'][df_clean_tokens['label'] == 0]))
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

wordcloud = WordCloud(width = 800, height = 800,
				background_color ='white',
				min_font_size = 10).generate(str(df_clean_tokens['text'][df_clean_tokens['label'] == 1]))
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
# Enseñamos las palabras más frecuentes de cada label
def get_top_n_words(corpus, n=None):
	vec = CountVectorizer().fit(corpus)
	bag_of_words = vec.transform(corpus)
	sum_words = bag_of_words.sum(axis=0)
	words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
	words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
	return words_freq[:n]

In [ ]:
# Top 20 palabras más frecuentes de cada label
print('Palabras más frecuentes en noticias Verdaderas:')
common_words = get_top_n_words(df_clean['text'][df_clean_tokens['label'] == 0], 20)
for word, freq in common_words:
	print(word, freq)
print('-------------')
print('Palabras más frecuentes en noticias Falsas:')
common_words = get_top_n_words(df_clean['text'][df_clean_tokens['label'] == 1], 20)
for word, freq in common_words:
	print(word, freq)

In [ ]:
# Dividimos los datos en train y test
p_train = 0.80

df_clean_tokens['is_train'] = np.random.uniform(0, 1, len(df)) <= p_train
df_train = df_clean_tokens[df_clean_tokens['is_train'] == True]
df_test =  df_clean_tokens[df_clean_tokens['is_train'] == False]
df_clean_tokens = df_clean_tokens.drop('is_train', 1)
df_train = df_train.drop('is_train', 1)
df_test = df_test.drop('is_train', 1)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para test: ", len(df_test))

In [ ]:
df_train.head(10)

## Caso 1: Sin N-Gramas

In [ ]:
# Preparamos los datos para procesar Apriori
te1 = TransactionEncoder()
te_ary1 = te1.fit(df_train['text']).transform(df_train['text'])
df_apriori1 = pd.DataFrame(te_ary1, columns=te1.columns_)
df_apriori1

In [ ]:
# Conjuntos de items frecuentes
itemsets_frecuentes1 = apriori(df_apriori1, min_support=0.2, use_colnames=True)
itemsets_frecuentes1

In [ ]:
# Reglas de asociación apriori
reglas1 = association_rules(itemsets_frecuentes1, metric="confidence", min_threshold=0.2)
reglas1

In [ ]:
# Escogemos las reglas que contienen solamente True o solamente False en el consecuente
reglas_VoF1 = reglas1[(reglas1['consequents'] == frozenset({'TRUE'})) | (reglas1['consequents'] == frozenset({'FALSE'}))]
reglas_VoF1.sort_values(by=['confidence'], ascending=False)

In [ ]:
# Vemos la confianza maxima de las reglas falsas y verdaderas por separado
reglas_F1 = reglas1[reglas1['consequents'] == frozenset({'FALSE'})]
max(reglas_F1['confidence'])

reglas_V1 = reglas1[reglas1['consequents'] == frozenset({'TRUE'})]
max(reglas_V1['confidence'])

In [ ]:
# Vemos las 10 reglas que tienen mayor confianza de las falsas
reglas_F1.sort_values(by=['confidence'], ascending=False).head(10)

In [ ]:
# Vemos las 10 reglas que tienen mayor confianza de las verdaderas
reglas_V1.sort_values(by=['confidence'], ascending=False).head(10)

## Caso 2: Con N-Gramas

In [ ]:
# Definimos get_grams
def get_ngrams(text, n):
	n_grams = ngrams(text, n)
	return [' '.join(grams) for grams in n_grams]

# Creamos un nuevo dataframe donde para cada noticia almacenamos los ngramas de 1 a 3 de longitud
df_train_ngramas = pd.DataFrame()
df_train_ngramas['text'] = df_train['text']
df_train_ngramas['ngrams'] = df_train_ngramas['text'].apply(lambda x: get_ngrams(x, 1)) + df_train_ngramas['text'].apply(lambda x: get_ngrams(x, 2)) + df_train_ngramas['text'].apply(lambda x: get_ngrams(x, 3))

df_test_ngramas = pd.DataFrame()
df_test_ngramas['text'] = df_test['text']
df_test_ngramas['label']	= df_test['label']
df_test_ngramas['ngrams'] = df_test_ngramas['text'].apply(lambda x: get_ngrams(x, 1)) + df_test_ngramas['text'].apply(lambda x: get_ngrams(x, 2)) + df_test_ngramas['text'].apply(lambda x: get_ngrams(x, 3))

# Descartamos los ngramas que incluyan las palabras 'TRUE' o 'FALSE'
df_train_ngramas['ngrams'] = df_train_ngramas['ngrams'].apply(lambda x: [ngram for ngram in x if 'TRUE' not in ngram and 'FALSE' not in ngram])
df_test_ngramas['ngrams'] = df_test_ngramas['ngrams'].apply(lambda x: [ngram for ngram in x if 'TRUE' not in ngram and 'FALSE' not in ngram])

In [ ]:
# Pasamos de listas ngramas a sets de ngramas, así no los tendremos repetidos
for i in range(len(df_train_ngramas['ngrams'])):
	df_train_ngramas['ngrams'].iloc[i] = set(df_train_ngramas['ngrams'].iloc[i])
 
for i in range(len(df_test_ngramas['ngrams'])):
	df_test_ngramas['ngrams'].iloc[i] = set(df_test_ngramas['ngrams'].iloc[i])

In [ ]:
# Creamos un set con todos los ngramas de train
ngramas = []
for i in range(len(df_train_ngramas['ngrams'])):
	ngramas += df_train_ngramas['ngrams'].iloc[i]
ngramas_set = set(ngramas)
len(ngramas_set)

In [ ]:
# Contar la frecuencia de cada ngrama en el dataframe con counter
ngramas_counter = Counter(ngramas)

# Extraemos en una lista los ngramas que aparecen menos de 'umbral' veces
umbral = 50
ngramas_eliminar = [ngrama for ngrama, freq in ngramas_counter.items() if freq < umbral]

# Calculamos cuáles no vamos a eliminar, restando los conjuntos
ngramas_no_eliminar = set(ngramas) - set(ngramas_eliminar)

In [ ]:
# Vemos el número total de ngramas en cada dataframe
print("Número total de ngramas en el dataframe original: ", len(ngramas_counter))
print("Número total de ngramas en el dataframe limpio: ", len(ngramas_no_eliminar))

In [ ]:
# Creamos un nuevo dataframe donde eliminamos los ngramas que aparecen menos de 'umbral' veces
df_train_ngramas_clean = pd.DataFrame()
df_train_ngramas_clean['text'] = df_train_ngramas['text']
df_train_ngramas_clean['ngrams'] = df_train_ngramas['ngrams'].apply(lambda x: set(x).intersection(ngramas_no_eliminar))

In [ ]:
# Nos quedamos con los ngramas maximales, es decir los ngramas que no están contenidos en otros ngramas
ngramas_maximales = set(ngramas_no_eliminar)
for ngrama in ngramas_no_eliminar:
	for ngrama2 in ngramas_no_eliminar:
		if ngrama != ngrama2 and ngrama in ngrama2:
			ngramas_maximales.discard(ngrama)
   			break
  
print("Número total de ngramas maximales: ", len(ngramas_maximales))

In [ ]:
# Creamos un set que contenga los bigramas incluidos en los ngramas maximales
bigramas_maximales = set()
for ngrama in ngramas_maximales:
	split = ngrama.split()
	if len(split) == 2:
		bigramas_maximales.add(ngrama)
	elif len(split) > 2:
		for i in range(len(split)-1):
			bigramas_maximales.add(split[i] + ' ' + split[i+1])

In [ ]:
# Creamos una lista de transacciones donde cada transacción es una lista de elementos
transactions = df_train_ngramas_clean['text'].tolist()

# Creamos un TransactionEncoder y lo ajustamos a la lista de transacciones
te = TransactionEncoder()
te.fit(transactions)

# Transformamos las transacciones en una matriz booleana donde las filas representan las transacciones y las columnas representan los elementos
data = te.transform(transactions)

# Creamos un nuevo DataFrame a partir de la matriz booleana y las columnas de elementos correspondientes
df_encoded = pd.DataFrame(data, columns=te.columns_)

In [ ]:
# Calculamos los itemsets frecuentes
freq_itemsets = apriori(df_encoded, min_support=0.25, use_colnames=True, verbose=1)
freq_itemsets

In [ ]:
# Reglas de asociación
reglas = association_rules(freq_itemsets, metric="confidence", min_threshold=0.2)
reglas

In [ ]:
# Escogemos las reglas que contienen un elemento en el antecedente y otro en el consecuente, y ese elemento es un unigrama
reglas1 = reglas[	(reglas['antecedents'].apply(lambda x: len(x)) == 1) & (reglas['consequents'].apply(lambda x: len(x)) == 1)	& (reglas['antecedents'].apply(lambda x: ' ' not in list(x)[0]) & reglas['consequents'].apply(lambda x: ' ' not in list(x)[0]))   ]

In [ ]:
reglas1

In [ ]:
# Creamos una función que diga si una cadena de caracteres es un substring de un ngrama maximal
def is_in_maximal(cadena):
	return cadena in bigramas_maximales

In [ ]:
# Escogemos las reglas cuyo antecedente y consecuente conjuntamente pertenecen a algun ngrama maximal
reglas2 = reglas1[	reglas1.apply(lambda x: is_in_maximal(list(x['antecedents'])[0] + ' ' + list(x['consequents'])[0]), axis=1)	]

In [ ]:
reglas2

In [ ]:
# Definimos las palabras maximales como aquellas que se encuentran en los ngramas maximales
palabras_maximales = set()
for ngrama in ngramas_maximales:
	split = ngrama.split()
	for palabra in split:
		palabras_maximales.add(palabra)

In [ ]:
# Escogemos las reglas que contienen solamente True o solamente False en el consecuente, y en el antecedente hay una palabra contenida en un ngrama maximal
reglas_V = reglas1[reglas1['consequents'] == frozenset({'TRUE'})]
reglas_V = reglas_V[reglas_V.apply(lambda x: list(x['antecedents'])[0] in palabras_maximales, axis=1)]
reglas_V.sort_values(by=['confidence'], ascending=False)
reglas_F = reglas1[reglas1['consequents'] == frozenset({'FALSE'})]
reglas_F = reglas_F[reglas_F.apply(lambda x: list(x['antecedents'])[0] in palabras_maximales, axis=1)]
reglas_F.sort_values(by=['confidence'], ascending=False)

# Agrupamos las reglas verdaderas y falsas en un solo dataframe
reglas_VoF = pd.DataFrame()
reglas_VoF = reglas_V.append(reglas_F)
reglas_VoF
reglas_VoF.sort_values(by=['confidence'], ascending=False)

In [ ]:
# Obtenemos una lista con los antecedentes de las reglas VoF
antecedentes_VoF = reglas_VoF['antecedents'].tolist()	

In [ ]:
reglas_V

In [ ]:
# Creamos una función que vea si hay una regla en reglas2 con el antecedente y consecuente que le pasamos
def is_in_reglas2(antecedente, consecuente):
	return (antecedente, consecuente) in reglas2.apply(lambda x: (list(x['antecedents'])[0], list(x['consequents'])[0]), axis=1).tolist()

In [ ]:
# Escogemos las palabras conectadas con True o False
palabras_conectadas = set()
nuevas = set([list(x)[0] for x in reglas_VoF['antecedents']])
while len(nuevas) > 0:
    palabras_conectadas_aux = nuevas.copy()
    nuevas = set()
    for ant in palabras_conectadas_aux:
        nuevas.update(set(reglas2[reglas2.apply(lambda x: list(x['consequents'])[0] == ant, axis=1)]['antecedents'].apply(lambda x: list(x)[0]).tolist()))
    nuevas.difference_update(palabras_conectadas)
    palabras_conectadas.update(nuevas)

In [ ]:
palabras_conectadas

In [ ]:
# Nos quedamos con las reglas donde tanto el antecedente como el consecuente son palabras conectadas
reglas3 = reglas2[reglas2.apply(lambda x: list(x['antecedents'])[0] in palabras_conectadas and list(x['consequents'])[0] in palabras_conectadas, axis=1)]

In [ ]:
# Eliminamos las reglas que forman ciclos al crear una red con las reglas3
G = nx.DiGraph()
	

In [ ]:
len(reglas3)

In [ ]:
max(reglas_V['confidence'])

In [ ]:
# Vemos las 10 reglas que tienen mayor confianza de las verdaderas
reglas_V.sort_values(by=['confidence'], ascending=False).head(10)

In [ ]:
len(reglas_F)

In [ ]:
max(reglas_F['confidence'])

In [ ]:
# Vemos las 10 reglas que tienen mayor confianza de las falsas
reglas_F.sort_values(by=['confidence'], ascending=False).head(10)

In [ ]:
min(reglas_F['support'])

### Grafo de Asociación

In [ ]:
# Seleccionamos las reglas que tienen una confianza mayor a 0.4
reglas_VoF_grafo = reglas_VoF[reglas_VoF['confidence'] > 0.4]
reglas_VoF_grafo = reglas_VoF_grafo.head(100)

# Creamos un grafo de las reglas, donde los nodos son los elementos y las aristas son las reglas, y el valor de la arista es la confianza
G = nx.DiGraph()

# Añadimos los nodos
for i in range(len(reglas3)):
	G.add_node(next(iter(reglas3['antecedents'].iloc[i])))
	G.add_node(next(iter(reglas3['consequents'].iloc[i])))

for i in range(len(reglas_VoF_grafo)):
	G.add_node(next(iter(reglas_VoF_grafo['antecedents'].iloc[i])))
	G.add_node(next(iter(reglas_VoF_grafo['consequents'].iloc[i])))
	
# Añadimos las aristas con la confianza como peso
for i in range(len(reglas3)):
	G.add_edge(next(iter(reglas3['antecedents'].iloc[i])), next(iter(reglas3['consequents'].iloc[i])), weight=reglas3['confidence'].iloc[i].round(2))

for i in range(len(reglas_VoF_grafo)):
	G.add_edge(next(iter(reglas_VoF_grafo['antecedents'].iloc[i])), next(iter(reglas_VoF_grafo['consequents'].iloc[i])), weight=reglas_VoF_grafo['confidence'].iloc[i].round(2))

# Dibujamos el grafo
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=0.5)
nx.draw(G, pos, with_labels=True, font_size=20, node_size=2000, node_color='lightblue', edge_color='black', width=1, alpha=0.7)
edge_labels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=20)
nx.draw_networkx_edges(G, pos, edgelist=G.edges(), edge_color='black', arrows=True, width=1, alpha=0.7, connectionstyle='arc3, rad=0.3')
plt.show()

In [ ]:
# Comprobamos si el grafo tiene ciclos
nx.is_directed_acyclic_graph(G)

### Calculo de Pesos de las Reglas

In [ ]:
df_test_ngrams.columns

In [ ]:
# Calculamos los pesos de las reglas mediante la confianza y de forma no lineal, para que las reglas con confianza alta tengan un peso mayor, las que tienen confianza 0.5 tengan peso nulo y las que tienen confianza baja tengan un peso negativo
reglas_VoF['peso'] = reglas_VoF['confidence'].apply(lambda x: (x - 0.5)**2)

# Calculamos los ngramas del conjunto test
df_test_ngrams_clean = pd.DataFrame()
df_test_ngrams_clean['text'] = df_test_ngrams['text']
df_test_ngrams_clean['label'] = df_test_ngrams['label']
df_test_ngrams_clean['ngrams'] = df_test_ngrams['ngrams'].apply(lambda x: set(x).intersection(ngramas_no_eliminar))

# Ahora que tenemos las reglas y los ngramas del conjunto test, vamos a clasificar cada noticia test según si cumplen o no las reglas y sumando los pesos de las reglas que se cumplen con votación positiva
df_test_ngrams_clean['votos'] = df_test_ngrams_clean['ngrams'].apply(lambda x: sum(reglas_VoF[reglas_VoF['antecedents'].isin(x)]['peso']))

In [ ]:
# Cambiamos False y True por 1 y 0
df_test_ngrams_clean['label'] = df_test_ngrams_clean['label'].apply(lambda x: 1 if x == True else 0)

In [ ]:
# Calculamos la predicción de cada noticia test
df_test_ngrams_clean['prediccion'] = df_test_ngrams_clean['votos'].apply(lambda x: 0 if x > 0 else 1)

# Calculamos la precisión
accuracy_score(df_test_ngrams_clean['label'], df_test_ngrams_clean['prediccion'])


In [ ]:
# Calculamos la matriz de confusión
confusion_matrix(df_test_ngrams_clean['label'], df_test_ngrams_clean['prediccion'])